# Advanced Features and MCP Usage

This notebook explores advanced Bermuda functionality including stochastic modeling, bootstrapping, and integration with MCP (Model Context Protocol) servers.

## Setup

In [ ]:
from installation import install
await install()

In [ ]:
import bermuda as tri
import numpy as np
import pandas as pd
import altair as alt
from datetime import date
import warnings
warnings.filterwarnings('ignore')

# Enable HTML rendering
alt.renderers.enable("html")

# Load the combined triangle from previous notebook
try:
    combined = tri.binary_to_triangle('data/excel/combined.trib')
    print(f"Loaded combined triangle: {len(combined)} cells")
except:
    # Fallback to individual triangles
    combined = tri.binary_to_triangle('data/excel/gl_filtered.trib')
    print(f"Loaded GL triangle: {len(combined)} cells")

## Stochastic Modeling with Bermuda

Bermuda's cell structure naturally supports stochastic modeling - each cell can hold arrays of simulated values.

In [ ]:
# Get the right edge for projection
right_edge = combined.right_edge
print(f"Right edge has {len(right_edge)} cells")

# Select a recent accident period for demonstration
recent_cells = [cell for cell in right_edge if cell.period_start.year >= 2014]
if recent_cells:
    demo_cell = recent_cells[0]
else:
    demo_cell = right_edge[-1]

print(f"\nDemo cell: {demo_cell.period_start} to {demo_cell.period_end}")
print(f"Current paid loss: ${demo_cell.values.get('paid_loss', 0):,.0f}")

In [ ]:
# Create stochastic projections for future development
np.random.seed(42)  # For reproducibility
n_simulations = 1000

# Simple example: project next year's development with uncertainty
current_paid = demo_cell.values.get('paid_loss', 1000000)

# Simulate development factors with lognormal distribution
# Mean factor of 1.15 (15% development) with 10% CV
mean_factor = 1.15
cv = 0.10
sigma = np.sqrt(np.log(1 + cv**2))
mu = np.log(mean_factor) - sigma**2 / 2

development_factors = np.random.lognormal(mu, sigma, n_simulations)
projected_values = current_paid * development_factors

print(f"Current value: ${current_paid:,.0f}")
print(f"\nProjected values (next evaluation):")
print(f"  Mean: ${np.mean(projected_values):,.0f}")
print(f"  Std Dev: ${np.std(projected_values):,.0f}")
print(f"  95th percentile: ${np.percentile(projected_values, 95):,.0f}")
print(f"  99th percentile: ${np.percentile(projected_values, 99):,.0f}")

In [ ]:
# Create new cells with stochastic values
stochastic_cells = []

for cell in recent_cells[:3]:  # Do 3 cells for demo
    current_paid = cell.values.get('paid_loss', 1000000)
    
    # Generate simulations
    simulated_values = current_paid * np.random.lognormal(mu, sigma, n_simulations)
    
    # Create new cell with array of values
    new_cell = tri.CumulativeCell(
        period_start=cell.period_start,
        period_end=cell.period_end,
        evaluation_date=tri.add_months(cell.evaluation_date, 12),
        values={'paid_loss_simulated': simulated_values},
        metadata=cell.metadata
    )
    stochastic_cells.append(new_cell)

# Create triangle with stochastic cells
stochastic_triangle = tri.Triangle(stochastic_cells)
print(f"Created stochastic triangle with {len(stochastic_triangle)} cells")
print(f"Each cell contains {n_simulations} simulations")

In [ ]:
# Aggregate statistics across stochastic cells
for cell in stochastic_cells:
    sims = cell.values['paid_loss_simulated']
    period_str = f"{cell.period_start.year}"
    
    print(f"\nPeriod {period_str} projections:")
    print(f"  Mean: ${np.mean(sims):,.0f}")
    print(f"  Median: ${np.median(sims):,.0f}")
    print(f"  75th %ile: ${np.percentile(sims, 75):,.0f}")
    print(f"  95th %ile: ${np.percentile(sims, 95):,.0f}")
    print(f"  VaR(95): ${np.percentile(sims, 95) - np.mean(sims):,.0f}")

## Bootstrapping

Bermuda includes bootstrapping functionality for resampling triangles.

In [ ]:
# Bootstrap the triangle for uncertainty analysis
from bermuda.utils import bootstrap

# Create a smaller triangle for bootstrapping demo
small_tri = combined.clip(
    min_period=date(2012, 1, 1),
    max_period=date(2014, 12, 31),
    max_eval=date(2016, 12, 31)
)

print(f"Original triangle: {len(small_tri)} cells")

# Bootstrap with replacement
n_bootstrap = 100
bootstrap_samples = []

for i in range(n_bootstrap):
    # Resample periods with replacement
    sampled_periods = np.random.choice(small_tri.periods, len(small_tri.periods), replace=True)
    
    # Filter triangle to sampled periods
    sampled_cells = []
    for period in sampled_periods:
        period_cells = [cell for cell in small_tri 
                       if cell.period_start == period[0] and cell.period_end == period[1]]
        sampled_cells.extend(period_cells)
    
    if sampled_cells:
        bootstrap_samples.append(tri.Triangle(sampled_cells))

print(f"Created {len(bootstrap_samples)} bootstrap samples")

In [ ]:
# Analyze bootstrap distribution
if bootstrap_samples:
    # Get total paid loss for each bootstrap sample
    bootstrap_totals = []
    for sample in bootstrap_samples:
        total = sum(cell.values.get('paid_loss', 0) for cell in sample.right_edge)
        bootstrap_totals.append(total)
    
    original_total = sum(cell.values.get('paid_loss', 0) for cell in small_tri.right_edge)
    
    print("Bootstrap Distribution of Total Paid Loss:")
    print(f"  Original: ${original_total:,.0f}")
    print(f"  Bootstrap Mean: ${np.mean(bootstrap_totals):,.0f}")
    print(f"  Bootstrap Std Dev: ${np.std(bootstrap_totals):,.0f}")
    print(f"  95% CI: ${np.percentile(bootstrap_totals, 2.5):,.0f} - ${np.percentile(bootstrap_totals, 97.5):,.0f}")

## Interactive Exercise: Berquist-Sherman Adjustment

The Berquist-Sherman method adjusts historical losses for changes in claim settlement patterns. This is crucial when claim handling practices have changed over time.

In [ ]:
# Setup for Berquist-Sherman adjustment
print("Berquist-Sherman Adjustment Exercise")
print("="*40)
print("\nThe Berquist-Sherman method adjusts for changes in:")
print("1. Claim settlement rates (paid BS)")
print("2. Case reserve adequacy (reported BS)")
print("\nFor this exercise, we'll implement a simplified version.")

In [ ]:
# TODO: Complete this Berquist-Sherman implementation
# This is a simplified version for educational purposes

def simplified_bs_adjustment(triangle, target_settlement_rate=0.95):
    """
    Simplified Berquist-Sherman adjustment for claim settlement patterns.
    
    TODO: Complete this function to:
    1. Calculate settlement rates for each development period
    2. Identify periods with different settlement patterns
    3. Adjust paid losses to reflect consistent settlement rate
    
    Args:
        triangle: Input triangle with paid_loss and reported_loss
        target_settlement_rate: Target settlement rate to adjust to
    
    Returns:
        Adjusted triangle
    """
    
    # TODO: Step 1 - Calculate settlement rates
    # Hint: settlement_rate = paid_loss / reported_loss
    
    # settlement_rates = {}
    # for cell in triangle:
    #     if 'reported_loss' in cell.values and cell.values['reported_loss'] > 0:
    #         rate = _____ / _____
    #         settlement_rates[cell] = rate
    
    # TODO: Step 2 - Identify adjustment factors
    # For cells with settlement rate different from target, calculate adjustment
    
    # adjustments = {}
    # for cell, rate in settlement_rates.items():
    #     if rate != target_settlement_rate:
    #         adjustment_factor = _____ / _____
    #         adjustments[cell] = adjustment_factor
    
    # TODO: Step 3 - Create adjusted cells
    # Apply adjustments to create new triangle
    
    # adjusted_cells = []
    # for cell in triangle:
    #     if cell in adjustments:
    #         new_values = cell.values.copy()
    #         new_values['paid_loss'] = _____
    #         adjusted_cell = tri.CumulativeCell(
    #             period_start=cell.period_start,
    #             period_end=cell.period_end,
    #             evaluation_date=cell.evaluation_date,
    #             values=new_values,
    #             metadata=cell.metadata
    #         )
    #         adjusted_cells.append(adjusted_cell)
    #     else:
    #         adjusted_cells.append(cell)
    
    # return tri.Triangle(adjusted_cells)
    
    print("Exercise: Complete the BS adjustment implementation above")
    return triangle

# Test the function
adjusted_triangle = simplified_bs_adjustment(combined)
print("\nOnce completed, this will show the adjusted triangle")

### Solution for Berquist-Sherman Exercise

In [ ]:
# Complete solution (hidden during workshop)
def simplified_bs_adjustment_solution(triangle, target_settlement_rate=0.95):
    """
    Simplified Berquist-Sherman adjustment - SOLUTION.
    """
    
    # Step 1 - Calculate settlement rates
    settlement_rates = {}
    for cell in triangle:
        if 'reported_loss' in cell.values and cell.values['reported_loss'] > 0:
            rate = cell.values.get('paid_loss', 0) / cell.values['reported_loss']
            settlement_rates[cell] = rate
    
    # Step 2 - Identify adjustment factors
    adjustments = {}
    for cell, rate in settlement_rates.items():
        if rate != target_settlement_rate and rate > 0:
            adjustment_factor = target_settlement_rate / rate
            adjustments[cell] = adjustment_factor
    
    # Step 3 - Create adjusted cells
    adjusted_cells = []
    for cell in triangle:
        if cell in adjustments:
            new_values = cell.values.copy()
            new_values['paid_loss'] = cell.values.get('paid_loss', 0) * adjustments[cell]
            new_values['bs_adjusted'] = True
            adjusted_cell = tri.CumulativeCell(
                period_start=cell.period_start,
                period_end=cell.period_end,
                evaluation_date=cell.evaluation_date,
                values=new_values,
                metadata=cell.metadata
            )
            adjusted_cells.append(adjusted_cell)
        else:
            adjusted_cells.append(cell)
    
    return tri.Triangle(adjusted_cells)

# Uncomment to see solution in action:
# adjusted = simplified_bs_adjustment_solution(combined, target_settlement_rate=0.90)
# print(f"Adjusted {len([c for c in adjusted if 'bs_adjusted' in c.values])} cells")

## Alternative Exercise: Data Quality Detection

Find anomalies or data quality issues in the triangle.

In [ ]:
def find_data_anomalies(triangle):
    """
    Find potential data quality issues in a triangle.
    
    TODO: Implement checks for:
    1. Negative incremental values
    2. Unreasonable development factors (>5x or <0.5x)
    3. Missing data patterns
    """
    
    anomalies = []
    
    # Check for negative incremental development
    inc_triangle = triangle.to_incremental()
    for cell in inc_triangle:
        for field, value in cell.values.items():
            if 'loss' in field and value < 0:
                anomalies.append({
                    'type': 'negative_incremental',
                    'period': cell.period_start,
                    'evaluation': cell.evaluation_date,
                    'field': field,
                    'value': value
                })
    
    # TODO: Add more checks
    # - Large development jumps
    # - Suspicious patterns (exact duplicates, round numbers)
    # - Inconsistent metadata
    
    return anomalies

# Run anomaly detection
anomalies = find_data_anomalies(combined)
print(f"Found {len(anomalies)} potential issues")
for anomaly in anomalies[:5]:  # Show first 5
    print(f"  {anomaly['type']}: {anomaly['field']} = {anomaly['value']:,.0f} at {anomaly['period']}")

## Summary

In this advanced session, we've explored:

1. **Stochastic Modeling**: Store and analyze arrays of simulated values in cells
2. **Bootstrapping**: Resample triangles for uncertainty quantification
3. **MCP Integration**: Leverage AI assistance for complex queries
4. **Berquist-Sherman**: Adjust for changing claim practices
5. **Data Quality**: Detect anomalies and issues

These advanced features demonstrate how Bermuda goes beyond simple triangle manipulation to support sophisticated actuarial analyses.

Key takeaways:
- Cells can hold complex data structures (arrays, distributions)
- Bermuda's functional approach enables Monte Carlo and bootstrap methods
- MCP integration can enhance productivity for complex analyses
- Standard actuarial adjustments can be implemented cleanly
- Data quality checks are essential for production systems